In [ ]:
#Getting set up
import pandas as pd #to handle the data frame
from sklearn.impute import SimpleImputer #to impute the missing values
from sklearn.preprocessing import OneHotEncoder # binarisation of the categorical attributes
from sklearn.preprocessing import StandardScaler #implements the z-score
from sklearn.decomposition import PCA #for dimensionality reduction after the transformations made
from sklearn import metrics #to evaluate the quality of clustering
import numpy as np #to convert the dataframe into muldimensional arrays and work with
import matplotlib.pyplot as plt #for plotting and visualization
import sklearn.cluster as cluster #Sklearn has 13 clustering algorithms

Most used and frequent libraries in clustering tasks

In [ ]:
df = pd.read_csv('CSVFile.csv')

In [ ]:
preprocess_num=df.loc[:,tayou_df.dtypes!=object]
preprocess_cat=df.loc[:,tayou_df.dtypes==object]

In [ ]:
#Preparing the data
#filling in the missing values
num_imputer = SimpleImputer(strategy="mean")
preprocess_num = df[num_vars].to_numpy()
preprocess_num=num_imputer.fit_transform(preprocess_num)
cat_imputer = SimpleImputer(strategy="most_frequent")
preprocess_cat = df[cat_vars].to_numpy()
preprocess_cat=cat_imputer.fit_transform(preprocess_cat)
#Handling categorical variables
cat_encoder = OneHotEncoder()
preprocess_cat = cat_encoder.fit_transform(preprocess_cat)
preprocess_cat = preprocess_cat.toarray()
print(preprocess_cat)

Deviding data into categorical and Numerical as they have different tratment

In [ ]:
preprocess_num=StandardScaler().fit_transform(preprocess_num)

Numerical Data normalization

In [ ]:
df_preprocessed=np.column_stack((preprocess_cat, preprocess_num))

Conncatination of Data after pre-process

In [ ]:
#Dimensionlaity reduction with PCA
pca = PCA(36, svd_solver="covariance_eigh")
df_reduced = pca.fit_transform(df_preprocessed)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

Reduce dimensinality to ensure that data is reduced so we can have better calculation time

In [ ]:
#Kmeans
#finidng the best k, number of clusters
#first method, elbow method
wcss = [] #initializing the variable that will contain the Within Cluster Sum of Squares computed at each iteration (The Y-axis of the curve)
max_clusters = min(10, len(df_reduced))
for i in range(1, max_clusters + 1): #to run the kmeans for different cluster numbers ranging from 1 to 10 (The X-axis of the curve)
    kmeans = cluster.KMeans(n_clusters=i, random_state=42)
    kmeans.fit(df_reduced)
    wcss.append(kmeans.inertia_)

# Plotting the Elbow Method graph
plt.plot(range(1, max_clusters + 1), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

Using the elbow methode to know what is the best number of clusters

The best number of clusters is simply the inflection point: https://www.google.com/imgres?q=elbow%20method%20for%20clustering&imgurl=https%3A%2F%2Fbuiltin.com%2Fsites%2Fwww.builtin.com%2Ffiles%2Fstyles%2Fckeditor_optimize%2Fpublic%2Finline-images%2F1_elbow-method.jpeg&imgrefurl=https%3A%2F%2Fbuiltin.com%2Fdata-science%2Felbow-method&docid=ddvzSng-N0TI6M&tbnid=-Lpc8W4OBEjPDM&vet=12ahUKEwiewa77m9WJAxVwTqQEHTneDzQQM3oECBsQAA..i&w=512&h=275&hcb=2&ved=2ahUKEwiewa77m9WJAxVwTqQEHTneDzQQM3oECBsQAA

POINT WHERE THE wcss gets stable

In [ ]:
#Kmeans
#finidng the best k, number of clusters
#first method, elbow method, second version with an indication
from yellowbrick.cluster import KElbowVisualizer
model = cluster.KMeans(random_state=42)
visualizer = KElbowVisualizer(model, k=(2,10))
visualizer.fit(df_reduced)
visualizer.show()

Using both WCSS and we can know the best score

In [ ]:
#Kmeans
#finidng the best k, number of clusters
#second method, silhouette coefficient
from yellowbrick.cluster import SilhouetteVisualizer
fig, ax = plt.subplots(3, 2, figsize=(15,8))
for i in [2, 3, 4, 5]:
#Create KMeans instances for different number of clusters
  km = cluster.KMeans(n_clusters=i, random_state=42)
  q, mod = divmod(i, 2)
#Create SilhouetteVisualizer instance with KMeans instance
#Fit the visualizer
  visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])
  visualizer.fit(df_reduced)

This gives us the best idea on how the data must be distrubuted (the best K ) as we calculate the Silhouette score. The best number of clusters is the one with the biggest Silhouette score

In [ ]:
#Initialize the class object
kmeans = cluster.KMeans(n_clusters= 3, random_state=42)

#predict the labels of clusters.
label = kmeans.fit_predict(df_reduced)

print(label)

Here you you will ned 3 into the conviniet number of clusters which we found after doing the previous operations

In [ ]:
#Plotting the clusters
#Getting unique labels
u_labels = np.unique(label)
#plotting the results:
for i in u_labels:
    plt.scatter(df_reduced[label == i , 0] , df_reduced[label == i , 1] , label = i)
plt.legend()
plt.show()

Disperssion of Data using visuail approch to know how our clusters have been created

In [ ]:
#Getting the Centroids
centroids = kmeans.cluster_centers_
u_labels = np.unique(label)

#plotting the results:

for i in u_labels:
    plt.scatter(df_reduced[label == i , 0] , df_reduced[label == i , 1] , label = i)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'k')
plt.legend()
plt.show()

Same as previous cell but it includes centroids to know the mean tendancy

In [ ]:
#evaluation of the clustering
print(f"Silhouette Coefficient for kmeans: {metrics.silhouette_score(df_reduced, label):.3f}")

Metric to be used Silhouette score

In [ ]:
#DBSCAN
#Finding the optimal parameters
# initialize the value of k for kNN which can be same as MinPts (rule of thumb: 2 * dimensionality of the data)
k = 4
# Compute k-nearest neighbors
# you need to add 1 to k as this function also return distance to itself
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=k+1).fit(df_reduced)

# get distances
dist, ind = nbrs.kneighbors(df_reduced)
#drop the first column, and sort distances in an ascending order
k_dist = np.sort(dist[:, -1])
#Plot the kNN graph with sorted distance,
plt.plot(k_dist)
plt.xlabel('Distance sorted points')
plt.ylabel(f'{k}-Distance')
plt.show()

In [ ]:
! pip install kneed
from kneed import KneeLocator
kneedle = KneeLocator(x = range(1, len(dist)+1), y = k_dist, S = 1.0,
                      curve = "concave", direction = "increasing", online=True)

# get the estimate of knee point
print(kneedle.knee_y)

In [ ]:
kneedle.plot_knee()
plt.show()

How to chose the parametersof The DSBCAN : a density based clustering algorithm that takes disperssion in consedration

In [ ]:
dbscan=cluster.DBSCAN(eps=1.25, min_samples=4, metric='euclidean')
label = dbscan.fit_predict(df_reduced)

Making DSBCAN predection , the hyper parameters will be detrmined using the 3 cells that preceed this one

In [ ]:
#Plotting the clusters
#Getting unique labels
u_labels = np.unique(label)
#plotting the results:
for i in u_labels:
    plt.scatter(df_reduced[label == i , 0] , df_reduced[label == i , 1] , label = i)
plt.legend()
plt.show()

Plot to see the differences

if there is a negative value for a cluster than that cluster is highly an outliers cluster ... meaning that it has elements that are far away from the general characterstic from our data

In [ ]:
#cluster evaluation
print(f"Silhouette Coefficient for DBSCAN: {metrics.silhouette_score(df_reduced, label):.3f}")

Score of this methode

In [ ]:
#Hierarchical clustering
#Agglomerative clustering
agglomerative = cluster.AgglomerativeClustering(n_clusters=3)
labels = agglomerative.fit_predict(df_reduced)
print(labels)

In [ ]:
from scipy.cluster.hierarchy import dendrogram
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram
    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
# setting distance_threshold=0 ensures we compute the full tree.
model = cluster.AgglomerativeClustering(distance_threshold=0, n_clusters=None) #Set the distance threshold or n_clusters

model = model.fit(df_reduced)
plt.title("Hierarchical Clustering Dendrogram")
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode="level", p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

This shows the dendrogram it considers all elements as seprate classes and than staart mixing themm until we have one class

There is a big misconception that we use it to deteect the right number of classes to use in KMEANS well it's a wrong approch so be careful

In [ ]:
#cluster evaluation
print(f"Silhouette Coefficient for agglomerative clustering: {metrics.silhouette_score(df_reduced, labels):.3f}")